In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re #정규식 지원
import csv #카테고리 cvs파일 활용
import datetime #파일명에 시간 활용

C:\Users\BIT\miniconda3\envs\ml-env\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (5.1.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
# 현재 날짜 시간

now = datetime.datetime.now()
current_time = now.strftime("%Y-%m-%d_%H-%M-%S")

print(now)

print(current_time)

2023-03-09 17:08:05.521620
2023-03-09_17-08-05


In [5]:
def yes24DataReader(CategoryNum, startPage, endPage):

    root_url = 'http://www.yes24.com'

    url_1 = 'http://www.yes24.com/24/Category/Display/'
    url_2 = '?PageNumber='
    url_set = url_1 + CategoryNum + url_2
    book_list=[]

    for i in range(startPage, endPage+1):

        url = url_set + str(i)
        html = requests.get(url).text
        soup = BeautifulSoup(html, 'lxml')
        url_tag = 'a.bgYUI.ico_nWin'
        books = soup.select(url_tag)

        # 수집 중인 페이지 번호 출력
        print('# Page', i)

        # 개별 도서 정보 수집
        for book in books:

            sub_url = root_url + book.get('href')
            sub_html = requests.get(sub_url).text
            sub_soup = BeautifulSoup(sub_html, 'lxml')            
#             print(sub_url)

            title_tag = '#yDetailTopWrap > div.topColRgt > div.gd_infoTop > div > h2'
            author_tag = '#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_pubArea > span.gd_auth > a' 
            author_tag2 = '#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_pubArea > span.gd_auth'
            publisher_tag = '#yDetailTopWrap > div.topColRgt > div.gd_infoTop > span.gd_pubArea > span.gd_pub > a'
            isbn_tag = '#infoset_specific > div.infoSetCont_wrap > div > table > tbody > tr:nth-child(3) > td'
            img_tag = 'img.gImg'
            category_tag = '#infoset_goodsCate > div.infoSetCont_wrap > dl > dd > ul > li:nth-child(1) > a:nth-child(4)'

            book_title = sub_soup.select(title_tag)[0].text.strip()
    
            
            try :
                book_author = sub_soup.select(author_tag)[0].text
            except :
                book_author = sub_soup.select(author_tag2)[0].text.replace("\r\n", "").replace(" ","")
            
            book_publisher = sub_soup.select(publisher_tag)[0].text
            
            try :
                book_isbn = sub_soup.find('th', text='ISBN13').find_next_sibling('td').text
            except :
                book_isbn = ''
                
            book_img = sub_soup.select(img_tag)[0].get('src')
#             book_category = 8
#             book_category = sub_soup.select(category_tag)[0].text

            category_mapping = {} # csv 파일 이용해서 매핑
            with open('./book_category_202303091301.csv', newline='', encoding='utf-8') as csvfile:
                reader = csv.DictReader(csvfile)
                for row in reader:
                    category_mapping[row['category']] = int(row['id']) # 카테고리 매핑 정보를 딕셔너리에 저장

            book_category = sub_soup.select(category_tag)[0].text # 웹에서 수집한 카테고리 정보 소설/시/희곡

            if book_category in category_mapping: # 매핑된 카테고리가 있다면
                book_category_id = category_mapping[book_category] # 카테고리를 숫자로 매핑
            else:
                book_category_id = 27 # 매핑된 카테고리가 없다면 None으로 처리

            book_list.append([book_title, book_author, book_publisher, book_isbn, book_img, book_category_id])

            print('=========>', book_title)

    # 데이터프레임 컬럼명 지정
    colList = ['book_title',  'book_author', 'book_publisher',  'book_ISBN', 'book_img_url', 'book_category_id']
    print(book_list)

    # 데이터프레임으로 변환
    df = pd.DataFrame(np.array(book_list), columns=colList)

    return df

In [7]:
CategoryNumber='017001045006'
startPage=1
endPage=1
df = yes24DataReader(CategoryNumber, startPage, endPage)
df

# Page 1
=========> [최근담] 꿀로 무거워져
=========> 비상문 - 테이크아웃 10
=========> 구의 증명 [할인]
=========> [최근담] 안온한 밤
=========> 불편한 편의점
=========> 혹시 MBTI가 어떻게 되세요?
=========> 아버지의 해방일지
=========> [최근담] 가꾸는 이의 즐거움
=========> 룬의 아이들 - 윈터러 완전판
=========> 사랑의 이해
=========> 불편한 편의점 2
=========> 당신이 찾아 헤매는 건 책이 아니야!
=========> 홍학의 자리
=========> [최근담] 하트모양 크래커
=========> [최근담] 이브와 트리
=========> 이토록 평범한 미래
=========> 2022 젊은작가상 신문
=========> [최근담] 가장 매혹적인
=========> [최근담] 만두 가게 앞에는 싱크홀이 있다
=========> [최근담] 그래머블 제로 Grammable zero
[['[최근담] 꿀로 무거워져', '김멜라', '예스이십사', '', 'http://image.yes24.com/goods/117201893/XL', 27], ['비상문 - 테이크아웃 10', '최진영', '미메시스', '9791155352205', 'http://image.yes24.com/goods/89869685/XL', 27], ['구의 증명 [할인]', '최진영', '은행나무', '9788956608594', 'http://image.yes24.com/momo/TopCate481/MidCate009/48084715(1).jpg', 27], ['[최근담] 안온한 밤', '백온유', '예스이십사', '', 'http://image.yes24.com/goods/117604359/XL', 27], ['불편한 편의점', '김호연', '나무옆의자', '9791161571195', 'http://image.yes24.com/goods/101506482/XL'

,book_title,book_author,book_publisher,book_ISBN,book_img_url,book_category_id
0,[최근담] 꿀로 무거워져,김멜라,예스이십사,,http://image.yes24.com/goods/117201893/XL,27
1,비상문 - 테이크아웃 10,최진영,미메시스,9791155352205,http://image.yes24.com/goods/89869685/XL,27
2,구의 증명 [할인],최진영,은행나무,9788956608594,http://image.yes24.com/momo/TopCate481/MidCate...,27
3,[최근담] 안온한 밤,백온유,예스이십사,,http://image.yes24.com/goods/117604359/XL,27
4,불편한 편의점,김호연,나무옆의자,9791161571195,http://image.yes24.com/goods/101506482/XL,27
5,혹시 MBTI가 어떻게 되세요?,정대건,읻다,9791189433680,http://image.yes24.com/goods/117173594/XL,27
6,아버지의 해방일지,정지아,창비,9788936495701,http://image.yes24.com/goods/112629320/XL,27
7,[최근담] 가꾸는 이의 즐거움,이유리,예스이십사,,http://image.yes24.com/goods/108737140/XL,27
8,룬의 아이들 - 윈터러 완전판,전민희 저,14월의책,9791190339018,http://image.yes24.com/goods/92697972/XL,27
9,사랑의 이해,이혁진,민음사,9788937439940,http://image.yes24.com/goods/72263335/XL,27
